In [1]:
from spykshrk.realtime.simulator import nspike_data
from spykshrk.realtime.ripple_process import RippleFilter, RippleParameterMessage
from spykshrk.realtime.realtime_base import NoneRecordBase

import multiprocessing as mp
import pandas as pd
pd.set_option('display.max_rows', 10)

In [2]:
%%time
bond_settings = {
    "base_dir": "/opt/data/daliu/other/mkarlsso/",
    "name": "bond",
    "days": [4],
    "epochs": [1], 
    "tetrodes": [5, 11, 12, 13, 14, 17, 19, 29]
}

anim = nspike_data.AnimalInfo(**bond_settings)

eeg = nspike_data.EEGDataStream(anim)

CPU times: user 17.3 s, sys: 576 ms, total: 17.9 s
Wall time: 17.7 s


In [3]:
eeg_df = eeg.data[4]

In [10]:

range = [24610253.0, 24610253.0 + 120 * 10000]
rip_param_dict = { "rip_coeff1": 1.2,
                   "rip_coeff2": 0.2,
                   "ripple_threshold": 5,
                   "samp_divisor": 10000,
                   "n_above_thresh": 1,
                   "lockout_time": 7500,
                   "detect_no_ripples": False,
                   "detect_no_ripple_time": 60000,
                   "dio_gate": False,
                   "dio_gate_port": -1,
                   "enabled": True,
                   "use_custom_baseline": True,
                   "update_custom_baseline": False }

rip_param_msg = RippleParameterMessage(**rip_param_dict)

rec_base = NoneRecordBase()

eeg_reduced_df = eeg_df.query('(index >= {}) & (index <= {})'.format(range[0], range[1]))

eeg_datas = []
for col_tet_num, eeg_col in eeg_reduced_df.items():
    eeg_col = eeg_col.dropna()
    
    eeg_datas.append(eeg_col)

rip_filters = []
for tet_id in eeg_df.columns:
    rip_filters.append(RippleFilter(rec_base=rec_base, param=rip_param_msg, ntrode_id=tet_id))


In [12]:
def run_ripple_filter(eeg_data):
    rip_param_dict = { "rip_coeff1": 1.2,
                   "rip_coeff2": 0.2,
                   "ripple_threshold": 5,
                   "samp_divisor": 50000,
                   "n_above_thresh": 1,
                   "lockout_time": 7500,
                   "detect_no_ripples": False,
                   "detect_no_ripple_time": 60000,
                   "dio_gate": False,
                   "dio_gate_port": -1,
                   "enabled": True,
                   "use_custom_baseline": True,
                   "update_custom_baseline": False }

    rip_param_msg = RippleParameterMessage(**rip_param_dict)

    rec_base = NoneRecordBase()
    
    rip_filt = RippleFilter(rec_base=rec_base, param=rip_param_msg, ntrode_id=-1)
    
    for eeg_time, eeg_val in eeg_data.items():
        rip_filt.process_data(eeg_time, eeg_val)
                
    return rip_filt.ripple_mean, rip_filt.ripple_std
    
p = mp.Pool(8)

p.map(run_ripple_filter, eeg_datas)

#p.map(run_ripple_filter, zip(rip_filters, eeg_datas))

[(12.769411875176933, 8.0341711293561211),
 (13.067350462251353, 8.1426347417953036),
 (10.124854354041736, 6.3962828908824783),
 (10.767028183929094, 6.8097532319051473),
 (13.015360171965858, 8.3150535639435947),
 (10.619217597888525, 6.5063986422966531),
 (12.996763007924329, 8.0572951859850992),
 (12.712036688440238, 8.0500970919242416)]